### Importing Required libraries:

In [ ]:
import numpy as np
from numpy.linalg import norm
import pickle
from tqdm import tqdm, tqdm_notebook
import os
import random
import time
import math
import tensorflow as tf
import numpy as np
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import PIL
from PIL import Image
from sklearn.neighbors import NearestNeighbors
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
import glob
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

### Loading the Resnet50 model:

In [ ]:
model = ResNet50(weights='imagenet',
                         include_top=False,
                         input_shape=(224, 224, 3),
                        pooling='max')

In [ ]:
#model.summary()



## Feature Extraction :

Let's define a function to extract image features given an image and a model. 

In [ ]:
def extract_features(img_path, model):
    input_shape = (224, 224, 3)
    img = image.load_img(img_path,target_size=(input_shape[0], input_shape[1]))
    img_array = image.img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img = preprocess_input(expanded_img_array)
    #extracting feature from model
    features = model.predict(preprocessed_img)
    flattened_features = features.flatten()
    normalized_features = flattened_features / norm(flattened_features)
    return normalized_features

Let's see the feature length the model generates by running on an example image. If you don't have the usual cat image available locally, let's download it!

In [ ]:
features = extract_features(IMG_PATH, model)
print("Total length of features for one image: ", len(features))

In [ ]:
%timeit features = extract_features(IMG_PATH, model)

## Benchmarking time taken to extract features over the entire dataset

The time taken to extract features is dependent on a few factors such as image size, computing power etc.

A better benchmark would be running the network over an entire dataset. 

A simple change to the existing code will allow this.

Let's make a handy function to recursively get all the image files under a root directory.

In [ ]:
extensions = ['.jpg', '.JPG', '.jpeg', '.JPEG', '.png', '.PNG']

def get_file_list(root_dir):
    file_list = []
    for root, directories, filenames in os.walk(root_dir):
        for filename in filenames:
            if any(ext in filename for ext in extensions):
                filepath = os.path.join(root, filename)
                if os.path.exists(filepath):
                    file_list.append(filepath)
                else:
                    print(filepath)
    return file_list

Now, let's run the extraction over the entire dataset and time it.

In [ ]:
# path to the your datasets
root_dir = r'D:\tensorflow\101_ObjectCategories\101_ObjectCategories'
filenames = sorted(get_file_list(root_dir))
print(len(filenames))

Now, let's run the extraction over the entire dataset and time it.

In [ ]:
BATCH_SIZE = 128
generator = tf.keras.utils.image_dataset_from_directory(root_dir,
                                            shuffle=False,
                                           batch_size=BATCH_SIZE,
                                            image_size=(224,224))

num_images = len(generator.file_paths)
num_epochs = int(math.ceil(num_images / BATCH_SIZE))

start_time = time.time()
feature_list = []
feature_list = model.predict(generator, num_epochs)
end_time = time.time()

In [ ]:
#d diimensional representation took about 16 min

In [ ]:
for i, features in enumerate(feature_list):
    feature_list[i] = features / norm(features)

feature_list = feature_list.reshape(len(feature_list), -1)

print("Num images   = ", len(generator.file_paths))
print("Shape of feature_list = ", feature_list.shape)
print("Time taken in min = ", (end_time - start_time)/60)

By now, we have generated features from the entire dataset of images. 

With the benchmarking experiments squared away, let's save the features as intermediate files to use later.

In [ ]:
pickle.dump(generator.class_names, open(r'C:\Users\ChethanNingappa\Downloads\experiment\class_names.txt','wb'))
pickle.dump(generator.file_paths, open(r'C:\Users\ChethanNingappa\Downloads\experiment\file_paths.txt', 'wb'))
pickle.dump(feature_list,open(r'C:\Users\ChethanNingappa\Downloads\experiment\resnet50' + '.pickle', 'wb'))

That’s all folks! We’re done with the feature extraction part

## Similarity Search:
- Given a QUERY image, our aim is to find another photo in our dataset similar to the current one. 

- We begin by loading the precomputed features:
 

In [ ]:
filenames = pickle.load(open(r'C:\Users\ChethanNingappa\Downloads\experiment\file_paths.txt', 'rb'))
feature_list = pickle.load(open(r"C:\Users\ChethanNingappa\Downloads\experiment\resnet50.pickle",'rb'))
class_ids = pickle.load(open(r'C:\Users\ChethanNingappa\Downloads\experiment\class_names.txt', 'rb'))

In [ ]:
num_images = len(filenames)
num_features_per_image = len(feature_list[0])
print("Number of images = ", num_images)
print("Number of features per image = ", num_features_per_image)

In [ ]:
feature_list.shape

In [ ]:
len(class_ids)

## How we find similar images after feature extraction ?

Using Knearest Neighbors :  
- We’ll use Python’s machine learning library scikit-learn for
finding nearest neighbors of the query features; that is, features that represent a query image. 

- We train a nearest-neighbor model using the brute-force algorithm to find the nearest five neighbors based on Euclidean distance

In [ ]:
random_index = random.randint(0, num_images)

In [ ]:
from sklearn.neighbors import NearestNeighbors

neighbors = NearestNeighbors(n_neighbors=5, algorithm='brute', metric='euclidean').fit(feature_list)
distances, indices = neighbors.kneighbors([feature_list[random_index]])

Now, let's look at the time it takes to search for the nearest neighbors for the selected random image using the trained model with the Brute force algorithm.

In [ ]:
%timeit distances, indices = neighbors.kneighbors([feature_list[random_index]])

In [ ]:
plt.imshow(mpimg.imread(filenames[random_index]))

In [ ]:
plt.imshow(mpimg.imread(filenames[indices[0][0]]))

In [ ]:
plt.imshow(mpimg.imread(filenames[indices[0][1]]))

In [ ]:
plt.imshow(mpimg.imread(filenames[indices[0][2]]))

In [ ]:
for i in range(5):
    print(distances[0][i])

In [ ]:
# Helper function to get the classname
def classname(str):
    return str.split('\\')[-2]


# Helper function to get the classname and filename
def classname_filename(str):
    return str.split('\\')[-2] + '\\' + str.split('\\')[-1]


# Helper functions to plot the nearest images given a query image
def plot_images(filenames, distances):
    images = []
    for filename in filenames:
        images.append(mpimg.imread(filename))
    plt.figure(figsize=(20, 10))
    columns = 4
    for i, image in enumerate(images):
        ax = plt.subplot(len(images) // columns + 1, columns, i + 1)
        if i == 0:
            ax.set_title("Query Image\n" + classname_filename(filenames[i]))
        else:
            ax.set_title("Similar Image\n" + classname_filename(filenames[i]) +
                         "\nDistance: " +
                         str(float("{0:.2f}".format(distances[i]))))
        plt.imshow(image)

In [ ]:
for i in range(10):
    random_image_index = random.randint(0, 8000)
    distances, indices = neighbors.kneighbors([feature_list[random_image_index]])
    
    # Don't take the first closest image as it will be the same image
    similar_image_paths = [filenames[random_image_index]] + \
        [filenames[indices[0][i]] for i in range(1, 4)]
#print(similar_image_paths)
    plot_images(similar_image_paths, distances[0].astype(int))

In [ ]:
# Calculating some stats
print("Median distance between all photos: ", np.median(distances))
print("Max distance between all photos: ", np.max(distances))
print("Median distance among most similar photos: ",np.median(distances[:, 2]))

### Calculating Accuracy of the Brute Force Model :

In [ ]:
# Helper function that calculates accuracy using the nearest neighbors brute force algorithm
def calculate_accuracy(feature_list):
    num_nearest_neighbors = 5
    correct_prediction = 0
    incorrect_prediction = 0
    neighbors = NearestNeighbors(n_neighbors=num_nearest_neighbors,algorithm='brute',metric='euclidean').fit(feature_list)
    start = time.time()
    for i in range(len(feature_list)):
        distances, indices = neighbors.kneighbors([feature_list[i]])
        for j in range(1, num_nearest_neighbors):
            if (classname(filenames[i]) == classname(filenames[indices[0][j]])):
                correct_prediction += 1
            else:
                incorrect_prediction += 1
    end = time.time()
    accuracy = round(100.0 * correct_prediction /
        (1.0 * correct_prediction + incorrect_prediction), 2), end - start
    return accuracy

In [ ]:
print("Accuracy on original feature set : ",calculate_accuracy(feature_list[:]))

## Visualizing Image Clusters with t-SNE:

Let’s step up the game by visualizing the entire dataset!

- To do this, we need to reduce the dimensions of the feature vectors because it’s not possible to plot a 2,048-dimension vector (the feature-length) in two dimensions (the paper).
- The t-distributed stochastic neighbor embedding (t-SNE) algorithm reduces the high-dimensional feature vector to 2D, providing a bird’s-eye view of the dataset, which is helpful in recognizing clusters and nearby images. 
- t-SNE is difficult to scale to large datasets, so it is a good idea to reduce the dimensionality using Principal Component Analysis (PCA) and then call t-SNE:

In [ ]:
# Perform PCA over the features
num_feature_dimensions=150 # Set the number of features 
pca = PCA(n_components = num_feature_dimensions)
pca.fit(feature_list)
feature_list_compressed = pca.transform(feature_list)

Time taken to search similar images for one image using PCA:

In [ ]:
neighbors = NearestNeighbors(n_neighbors=5, algorithm='brute', metric='euclidean').fit(feature_list_compressed)
distances, indices = neighbors.kneighbors([feature_list_compressed[random_index]])
%timeit distances, indices = neighbors.kneighbors([feature_list_compressed[random_index]])

In [ ]:
# For speed and clarity, we'll analyze about first half of the dataset.
selected_features = feature_list_compressed[:4000]
selected_class_ids = class_ids[:4000]
selected_filenames = filenames[:4000]
tsne_results = TSNE(n_components=2,verbose=1,metric='euclidean').fit_transform(selected_features)
# Plot a scatter plot from the generated t-SNE results
colormap = plt.cm.get_cmap('coolwarm')
scatter_plot = plt.scatter(tsne_results[:,0],tsne_results[:,1],c=None, cmap=colormap)
plt.colorbar(scatter_plot)
plt.show()

In [ ]:
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from matplotlib.cbook import get_sample_data


def plot_images_in_2d(x, y, image_paths, axis=None, zoom=1):
    if axis is None:
        axis = plt.gca()
    x, y = np.atleast_1d(x, y)
    for x0, y0, image_path in zip(x, y, image_paths):
        image = Image.open(image_path)
        image.thumbnail((100, 100), Image.ANTIALIAS)
        img = OffsetImage(image, zoom=zoom)
        anno_box = AnnotationBbox(img, (x0, y0),xycoords='data',frameon=False)
        axis.add_artist(anno_box)
    axis.update_datalim(np.column_stack([x, y]))
    axis.autoscale()

In [ ]:
def show_tsne(x, y, selected_filenames):
    fig, axis = plt.subplots()
    fig.set_size_inches(22, 22, forward=True)
    plot_images_in_2d(x, y, selected_filenames, zoom=0.3, axis=axis)
    plt.show()

In [ ]:
show_tsne(tsne_results[:, 0], tsne_results[:, 1], selected_filenames)


Neat! There is a clearly demarcated cluster of human faces, flowers, vintage cars, ships, bikes, and a somewhat spread-out cluster of land and marine animals. 

## Accuracy of Brute Force over the PCA compressed Caltech101 features


In [ ]:
pca_dimensions = [1, 2, 3, 4, 5, 10, 20, 50, 75, 100, 150, 200]
pca_accuracy = []
pca_time = []

for dimensions in pca_dimensions:
    pca = PCA(n_components=dimensions)
    pca.fit(feature_list)
    feature_list_compressed = pca.transform(feature_list[:])
    # Calculate accuracy over the compressed features
    accuracy, t = calculate_accuracy(feature_list_compressed[:])
    pca_time.append(t)
    pca_accuracy.append(accuracy)
    print("For PCA Dimensions = ", dimensions, ",\tAccuracy = ", accuracy, "%",
          ",\tTime = ", pca_time[-1])

### PCA + Annoy:

- Annoy (Approximate Nearest Neighbors Oh Yeah) is a C++ library with Python bindings for searching nearest neighbors. 
- Synonymous with speed, it was released by Spotify and is used in production to serve their music recommendations.

- To use annoy, install it using pip: pip3 install annoy

In [ ]:
!pip install annoy

In [ ]:
from annoy import AnnoyIndex

First, we build a search index with two hyperparameters - the number of dimensions of the dataset, and the number of trees.

In [ ]:
# Time the indexing for Annoy
t = AnnoyIndex(150)  # Length of item vector that will be indexed
starttime = time.time()
for i in range(num_images):
    feature = feature_list_compressed[i]
    t.add_item(i, feature)
endtime = time.time()
print(endtime - starttime)
t.build(40)  # 40 trees
t.save('data/caltech101index.ann')

Annoy on one image:
Now let’s find out the time it takes to search the 5 nearest neighbors of one image.

In [ ]:
random_image_index = 1001

In [ ]:
u = AnnoyIndex(150)
%timeit u.get_nns_by_vector(feature_list_compressed[random_image_index], 5, include_distances=True)
indexes = u.get_nns_by_vector(feature_list_compressed[random_image_index],
                              5,
                              include_distances=True)

In [ ]:
def calculate_annoy_time():
    for i in range(0, 100):
        indexes = u.get_nns_by_vector(feature_list_compressed[random_image_index],5,include_distances=True)

Annoy on a set of images:

Time the search for multiple images for Annoy.

In [ ]:
%time calculate_annoy_time()

## Conclusion:

Feature extraction using RESNET50 on Caltech101 dataset:
- Bruteforce + 2048 length feature:59.8ms
- Bruteforce + PCA(150):5.3ms
- Annoy + PCA(150):19us 